In [1]:
import pandas as pd

wl = pd.read_csv("wordsim353crowd.csv")
wl.head()

,Word 1,Word 2,Human (Mean)
0,admission,ticket,5.5360
1,alcohol,chemistry,4.1250
2,aluminum,metal,6.6250
3,announcement,effort,2.0625
4,announcement,news,7.1875


In [2]:
len(wl.index)

353

In [118]:
wl_low_sim = wl[wl["Human (Mean)"] <= 4].copy().reset_index(drop=True)[["Word 1", "Word 2"]]
wl_low_sim.head()

,Word 1,Word 2
0,announcement,effort
1,announcement,production
2,Arafat,Jackson
3,Arafat,peace
4,Arafat,terror


In [4]:
wl_low_sim.to_csv("custom_word_pairs.csv", index=False)

In [6]:
# Use a pipeline as a high-level helper
from huggingface_hub import login
from transformers import pipeline

login()

In [7]:
pipe = pipeline("text-generation", model="meta-llama/Llama-3.1-8B-Instruct")

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

Device set to use cuda:0


In [119]:
prompts = [
    f"Generate a funny joke that contains the following two words: `{w1}` and `{w2}`. Return only the joke. \n\n"
    for (w1, w2) in zip(wl_low_sim["Word 1"], wl_low_sim["Word 2"])
]
prompts[0]

'Generate a funny joke that contains the following two words: `announcement` and `effort`. Return only the joke. \n\n'

In [142]:
import random
from nltk.corpus import wordnet as wn

import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')

# Get a list of common nouns
nouns = [w.name().split('.')[0] for w in wn.all_synsets('n')]
nouns = list(set(nouns))  # remove duplicates

def wordnet_similarity(w1, w2):
    syns1 = wn.synsets(w1)
    syns2 = wn.synsets(w2)
    if not syns1 or not syns2:
        return 0
    # Take max path similarity among all combinations
    sims = [s1.path_similarity(s2) for s1 in syns1 for s2 in syns2 if s1.path_similarity(s2)]
    return max(sims) if sims else 0

def random_low_similarity_pairs(n=10, threshold=0.2):
    pairs = []
    while len(pairs) < n:
        w1, w2 = random.sample(nouns, 2)
        sim = wordnet_similarity(w1, w2)
        if sim < threshold:
            w1 = w1.replace("_", " ")
            w2 = w2.replace("_", " ")
            pairs.append((w1, w2))
    return pairs

[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/jovyan/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [144]:
pairs = random_low_similarity_pairs(5000)

In [145]:
prompts = [
    f"Generate a funny joke that contains the following two words: `{w1}` and `{w2}`. Return only the joke. \n\n"
    for (w1, w2) in pairs
]
prompts[0]

'Generate a funny joke that contains the following two words: `silphium` and `snakeblenny`. Return only the joke. \n\n'

In [146]:
from torch.utils.data import Dataset
from transformers import AutoTokenizer

class ListDataset(Dataset):
    def __init__(self, original_list):
        self.original_list = original_list
        self.tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.1-8B-Instruct")

    def __len__(self):
        return len(self.original_list)

    def __getitem__(self, i):
        messages = [{"role": "user", "content": self.original_list[i]}]
        return self.tokenizer.apply_chat_template(messages, tokenize=False)

In [147]:
prompt_dataset = ListDataset(prompts)
generated_jokes = []
for out in tqdm(pipe(prompt_dataset, pad_token_id=pipe.tokenizer.eos_token_id, max_new_tokens=64)):
    response = out[0]["generated_text"].split("<|eot_id|>assistant\n\n")[1]
    generated_jokes.append(response)

100%|██████████| 5000/5000 [1:21:27<00:00,  1.02it/s]


In [148]:
generated_jokes[5]

'Why did the Bairdiella fish go to the party with the ocean expert? Because it wanted to meet a bigger knower of the sea.'

In [149]:
obs_prompt = """
You are a person who enjoys observational humour. 
Observational jokes are an examination of everyday things or situations through a comedic lens. 
Observational comedy covers topics familiar to almost everyone, even the most trivial aspects of life.
Do you think the following joke is funny or boring? {}
Reply either `It is funny.` or `It is boring.` followed by a brief justification.
"""

In [150]:
anec_prompt = """
You are a person who enjoys anecdotal humour.
Anecdotal humor is pulled from the comedian’s personal life and is popular with audiences because we can identify with their stories. 
Writer, producer and director Judd Apatow, who also performs stand-up comedy, believes that stand-up gets better as it becomes more personal—that comics who lay themselves bare to the audience are often the strongest performers. 
He gives the following example: one of his daughters has gone to college. 
His remaining daughter is unhappy that she is the only one left in the house with Judd and his wife, because four people is a family, but three people is a child observing a weird couple. 
You get the most laughs when the audience recognizes themselves in your story or joke.
Do you think the following joke is funny or boring? {}
Reply either `It is funny.` or `It is boring.` followed by a brief justification.
"""

In [151]:
onel_prompt = """
You are a person who enjoys One-liners. 
“I’ve had a perfectly wonderful evening, but this wasn’t it.”
That one-liner was delivered by Groucho Marx. 
Robin Williams once joked, “Why do they call it rush hour when nothing moves?” 
One-liners squeeze a setup and a punchline into one succinct thought.
Do you think the following joke is funny or boring? {}
Reply either `It is funny.` or `It is boring.` followed by a brief justification.
"""

In [152]:
iro_prompt = """
You are a person who enjoys Ironic jokes. 
Ironic jokes are contradictory, with two opposing concepts tugging at one another. 
For example, why do people park in a driveway but drive on a parkway?
Do you think the following joke is funny or boring? {}
Reply either `It is funny.` or `It is boring.` followed by a brief justification.
"""

In [153]:
self_def_prompt = """
You are a person who enjoys Self-deprecating humour. 
Some comedians make fun of the person they know best—themselves. 
Rodney Dangerfield made a career of self-deprecating jokes, poking fun at his looks and his love life with jokes like this: 
“I went to the psychiatrist, and he says ‘You're crazy.’ 
I tell him I want a second opinion. 
He says, ‘Okay, you're ugly too!’”
Do you think the following joke is funny or boring? {}
Reply either `It is funny.` or `It is boring.` followed by a brief justification.
"""

In [154]:
obs_prompts = [obs_prompt.format(jk) for jk in generated_jokes]
obs_prompts = ListDataset(obs_prompts)
obs_evals = []
for out in tqdm(pipe(obs_prompts, pad_token_id=pipe.tokenizer.eos_token_id, max_new_tokens=64)):
    response = out[0]["generated_text"].split("<|eot_id|>assistant\n\n")[1]
    obs_evals.append(response)

IOPub message rate exceeded.1:28:11<42:10,  1.57s/it]  
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [155]:
print(generated_jokes[0])
print("\nEval:")
print(obs_evals[2])

Why did the silphium farmer's pet snakeblenny go to therapy? Because it had a hiss-terical problem with its roots.

Eval:
It is funny. 

This joke is funny because it's an unexpected twist on a common concept (punching cards) and applies it to a specific situation (the Hollerith card) and then takes it to an absurd level by introducing the idea of a "jumping orchid" which


In [156]:
anec_prompts = [anec_prompt.format(jk) for jk in generated_jokes]
anec_prompts = ListDataset(anec_prompts)
anec_evals = []
for out in tqdm(pipe(anec_prompts, pad_token_id=pipe.tokenizer.eos_token_id, max_new_tokens=64)):
    response = out[0]["generated_text"].split("<|eot_id|>assistant\n\n")[1]
    anec_evals.append(response)

IOPub message rate exceeded.1:18:21<52:33,  1.58s/it]  
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

100%|██████████| 5000/5000 [2:10:36<00:00,  1.57s/it]


In [157]:
anec_evals[5]

'It is boring. \n\nThis joke relies on a play on words with "knower of the sea" and "know a bigger sea," but it\'s a one-liner that lacks personal connection or relatability. It doesn\'t draw from the comedian\'s own life or experiences, which makes it'

In [158]:
onel_prompts = [onel_prompt.format(jk) for jk in generated_jokes]
onel_prompts = ListDataset(onel_prompts)
onel_evals = []
for out in tqdm(pipe(onel_prompts, pad_token_id=pipe.tokenizer.eos_token_id, max_new_tokens=64)):
    response = out[0]["generated_text"].split("<|eot_id|>assistant\n\n")[1]
    onel_evals.append(response)

IOPub message rate exceeded.1:14:59<50:28,  1.51s/it]  
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [159]:
iro_prompts = [iro_prompt.format(jk) for jk in generated_jokes]
iro_prompts = ListDataset(iro_prompts)
iro_evals = []
for out in tqdm(pipe(iro_prompts, pad_token_id=pipe.tokenizer.eos_token_id, max_new_tokens=64)):
    response = out[0]["generated_text"].split("<|eot_id|>assistant\n\n")[1]
    iro_evals.append(response)

IOPub message rate exceeded.1:17:35<45:05,  1.35s/it]  
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [160]:
sd_prompts = [self_def_prompt.format(jk) for jk in generated_jokes]
sd_prompts = ListDataset(sd_prompts)
sf_evals = []
for out in tqdm(pipe(sd_prompts, pad_token_id=pipe.tokenizer.eos_token_id, max_new_tokens=64)):
    response = out[0]["generated_text"].split("<|eot_id|>assistant\n\n")[1]
    sf_evals.append(response)

100%|██████████| 5000/5000 [2:08:33<00:00,  1.54s/it]  


In [161]:
eval_df = pd.DataFrame()
eval_df["joke"] = generated_jokes
eval_df["anecdotal"] = [s.split()[2].rstrip('.') for s in anec_evals]
eval_df["observational"] = [s.split()[2].rstrip('.') for s in obs_evals]
eval_df["one_liner"] = [s.split()[2].rstrip('.') for s in onel_evals]
eval_df["irony"] = [s.split()[2].rstrip('.') for s in iro_evals]
eval_df["self_deprecating"] = [s.split()[2].rstrip('.') for s in sf_evals]

In [162]:
eval_df["score"] = eval_df.apply(
    lambda row: sum(str(cell).lower().count('funny') for cell in row[1:]),
    axis=1
)
eval_df.head()

,joke,anecdotal,observational,one_liner,irony,self_deprecating,score
0,Why did the silphium farmer's pet snakeblenny ...,boring,funny,funny,funny,funny,4
1,Why did the basal body temperature go to thera...,boring,funny,funny,funny,funny,4
2,Why did the hollerith card go to therapy after...,boring,funny,funny,funny,boring,3
3,Why did the Tocharian singer break up with his...,boring,funny,funny,funny,funny,4
4,"Why did the cormorant, a member of the Phalacr...",funny,funny,funny,funny,funny,5


In [163]:
eval_df.head()

,joke,anecdotal,observational,one_liner,irony,self_deprecating,score
0,Why did the silphium farmer's pet snakeblenny ...,boring,funny,funny,funny,funny,4
1,Why did the basal body temperature go to thera...,boring,funny,funny,funny,funny,4
2,Why did the hollerith card go to therapy after...,boring,funny,funny,funny,boring,3
3,Why did the Tocharian singer break up with his...,boring,funny,funny,funny,funny,4
4,"Why did the cormorant, a member of the Phalacr...",funny,funny,funny,funny,funny,5


In [164]:
eval_df.to_csv("eval_data.csv", index=False)

In [203]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModel
from tqdm import tqdm

class EvalDataset(Dataset):
    
    def __init__(self, eval_df):
        self.eval_df = eval_df.copy()
        
        # get the 5 individual joke category evals
        self.label_columns = ['anecdotal', 'observational', 'one_liner', 'irony', 'self_deprecating']
        self.eval_df[self.label_columns] = self.eval_df[self.label_columns].replace({'funny': 1, 'boring': 0})
        self.eval_df = self.eval_df[self.eval_df[self.label_columns].apply(lambda x: x.isin([0, 1]).all(), axis=1)]
        
        self.labels = self.eval_df[self.label_columns].to_dict(orient='records')
        self.jokes = self.eval_df.joke.to_list()
        
        # load tokenizer and model
        self.tokenizer = AutoTokenizer.from_pretrained("FacebookAI/xlm-roberta-large")
        self.model = AutoModel.from_pretrained("FacebookAI/xlm-roberta-large").to("cuda")
        self.model.eval()
        
        # generate embeddings
        self.features = self._featurize_all()
    
    def _featurize_all(self):
        features = []
        device = next(self.model.parameters()).device
        
        for text in tqdm(self.jokes):
            with torch.no_grad():
                tokens = self.tokenizer(text, return_tensors="pt", truncation=True, 
                                       padding=True, max_length=128).to(device)
                outputs = self.model(**tokens)
                cls_embedding = outputs.last_hidden_state[:, 0, :].squeeze(0)
                features.append(cls_embedding.cpu())
        
        return torch.stack(features)
    
    def __getitem__(self, i):
        features = self.features[i]
        labels_dict = {
            'anecdotal': torch.tensor(self.labels[i].get('anecdotal', 0), dtype=torch.long),
            'observational': torch.tensor(self.labels[i].get('observational', 0), dtype=torch.long),
            'ironic': torch.tensor(self.labels[i].get('irony', 0), dtype=torch.long),
            'one_liner': torch.tensor(self.labels[i].get('one_liner', 0), dtype=torch.long),
            'self_deprecating': torch.tensor(self.labels[i].get('self_deprecating', 0), dtype=torch.long)
        }
        
        return {"features": features, "labels": labels_dict}
    
    def __len__(self):
        return len(self.eval_df)

In [204]:
import torch.nn as nn
import torch.nn.functional as F

class JokeEvaluationModel(nn.Module):
    
    def __init__(self, num_labels=5, embedding_dim=768):
        super().__init__()
        self.shared_fc = nn.Linear(embedding_dim, 512)
        self.drop = nn.Dropout(0.3)

        self.anec_clf = nn.Linear(512, 2)
        self.obs_clf = nn.Linear(512, 2)
        self.iro_clf = nn.Linear(512, 2)
        self.onel_clf = nn.Linear(512, 2)
        self.sd_clf = nn.Linear(512, 2)

    def forward(self, features, labels, **kwargs):

        x = self.drop(features)
        x = self.shared_fc(x)
        x = F.relu(x)

        x_a = self.anec_clf(x)
        x_ob = self.obs_clf(x)
        x_iro = self.iro_clf(x)
        x_on = self.onel_clf(x)
        x_sd = self.sd_clf(x)

        return x_a, x_ob, x_iro, x_on, x_sd
    

In [205]:
from sklearn.model_selection import train_test_split

# Split into train (70%) and temp (30%)
train_df, temp_df = train_test_split(eval_df, test_size=0.3, random_state=42)

# Split temp into val (15%) and test (15%)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

# Create datasets
train_dataset = EvalDataset(train_df)
val_dataset = EvalDataset(val_df)
test_dataset = EvalDataset(test_df)

# Create dataloaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

/tmp/ipykernel_68/1844628415.py:13: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  self.eval_df[self.label_columns] = self.eval_df[self.label_columns].replace({'funny': 1, 'boring': 0})
100%|██████████| 3494/3494 [00:19<00:00, 177.66it/s]
/tmp/ipykernel_68/1844628415.py:13: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  self.eval_df[self.label_columns] = self.eval_df[self.label_columns].replace({'funny': 1, 'boring': 0})
100%|██████████| 750/750 [00:04<00:00, 177.05it/s]
/tmp/ipykernel_68/1844628415.py:13: FutureWarning: Downcasting

In [208]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader


def train_epoch(model, train_loader, optimizer, device):
    model.train()
    total_loss = 0
    
    for batch in tqdm(train_loader):
        features = batch["features"].to(device)
        labels = batch["labels"]
        
        # move all labels to device
        anec_labels = labels["anecdotal"].to(device)
        obs_labels = labels["observational"].to(device)
        iro_labels = labels["ironic"].to(device)
        onel_labels = labels["one_liner"].to(device)
        sd_labels = labels["self_deprecating"].to(device)
        
        optimizer.zero_grad()
        
        # forward pass
        x_a, x_ob, x_iro, x_on, x_sd = model(features=features, labels=labels)
        
        # compute losses for each classifier
        criterion = nn.CrossEntropyLoss()
        loss_a = criterion(x_a, anec_labels)
        loss_ob = criterion(x_ob, obs_labels)
        loss_iro = criterion(x_iro, iro_labels)
        loss_on = criterion(x_on, onel_labels)
        loss_sd = criterion(x_sd, sd_labels)
        
        # total loss -> sum of all classifier losses
        total_batch_loss = loss_a + loss_ob + loss_iro + loss_on + loss_sd
        
        # backward pass
        total_batch_loss.backward()
        optimizer.step()
        
        total_loss += total_batch_loss.item()
    
    avg_loss = total_loss / len(train_loader)
    return avg_loss


def evaluate(model, val_loader, device):
    model.eval()
    total_loss = 0
    
    with torch.no_grad():
        for batch in tqdm(val_loader):
            features = batch["features"].to(device)
            labels = batch["labels"]
            
            anec_labels = labels["anecdotal"].to(device)
            obs_labels = labels["observational"].to(device)
            iro_labels = labels["ironic"].to(device)
            onel_labels = labels["one_liner"].to(device)
            sd_labels = labels["self_deprecating"].to(device)
            
            x_a, x_ob, x_iro, x_on, x_sd = model(features=features, labels=labels)
            
            criterion = nn.CrossEntropyLoss()
            loss_a = criterion(x_a, anec_labels)
            loss_ob = criterion(x_ob, obs_labels)
            loss_iro = criterion(x_iro, iro_labels)
            loss_on = criterion(x_on, onel_labels)
            loss_sd = criterion(x_sd, sd_labels)
            
            total_batch_loss = loss_a + loss_ob + loss_iro + loss_on + loss_sd
            total_loss += total_batch_loss.item()
    
    avg_loss = total_loss / len(val_loader)
    return avg_loss


def train(model, train_loader, val_loader, epochs=10, learning_rate=1e-4, device='cuda'):
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    best_val_loss = float('inf')
    
    model.to(device)
    
    for epoch in range(epochs):
        train_loss = train_epoch(model, train_loader, optimizer, device)
        val_loss = evaluate(model, val_loader, device)
        
        print(f"Epoch {epoch + 1}/{epochs}")
        print(f"  Train Loss: {train_loss:.4f}")
        print(f"  Val Loss: {val_loss:.4f}")
        
        # save best model
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), "best_model.pt")
    
    print(f"\nTraining complete. Best validation loss: {best_val_loss:.4f}")
    return model

In [209]:
model = JokeEvaluationModel(embedding_dim=1024).to("cuda")
train(model, train_loader, val_loader, epochs=20, learning_rate=1e-4)

100%|██████████| 24/24 [00:00<00:00, 1137.30it/s]


Epoch 1/20
  Train Loss: 1.5053
  Val Loss: 1.1763


100%|██████████| 24/24 [00:00<00:00, 1144.90it/s]


Epoch 2/20
  Train Loss: 1.1476
  Val Loss: 1.1675


100%|██████████| 24/24 [00:00<00:00, 1163.31it/s]


...

100%|██████████| 24/24 [00:00<00:00, 1156.16it/s]


Epoch 19/20
  Train Loss: 1.0596
  Val Loss: 1.1609


100%|██████████| 24/24 [00:00<00:00, 1158.98it/s]

Epoch 20/20
  Train Loss: 1.0564
  Val Loss: 1.1084

Training complete. Best validation loss: 1.0904


JokeEvaluationModel(
  (shared_fc): Linear(in_features=1024, out_features=512, bias=True)
  (drop): Dropout(p=0.3, inplace=False)
  (anec_clf): Linear(in_features=512, out_features=2, bias=True)
  (obs_clf): Linear(in_features=512, out_features=2, bias=True)
  (iro_clf): Linear(in_features=512, out_features=2, bias=True)
  (onel_clf): Linear(in_features=512, out_features=2, bias=True)
  (sd_clf): Linear(in_features=512, out_features=2, bias=True)
)